<a href="https://colab.research.google.com/github/eemlcommunity/PracticalSessions2023/blob/omardd%2Frl/reinforcement_learning/introduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [EEML 2023] RL Tutorial - Introduction



In reinforcement learning (RL), we are interested in learning **policies** that allow an agent to **act optimally** in an **environment**.

There are two fundamental concepts in RL:

* policies $\pi(s, a)$ that prescribes with action $a$ to take in a state $s$;
* value functions $Q(s, a)$ that represent the value of taking action $a$ in a state $s$.

An optimal policy $\pi^*$ can be obtained from the optimal Q function $Q^*$ by selecting in each state $s$ the action maximizing $Q^*(s, a)$.


This tutorial focuses on **value-based** algorithms, i.e., those aiming to approximate $Q^*$.

## Interacting with an environment

We are going to use the [Gymnasium API](https://gymnasium.farama.org/) to create and interact with RL environments in Python.

In [ ]:
!pip install -q swig > /dev/null 2>&1
!pip install "gymnasium[box2d]" > /dev/null 2>&1

# install rlberry library (https://github.com/rlberry-py/rlberry)
!pip install rlberry==0.5.0 > /dev/null 2>&1
# install ffmpeg-python for saving videos
!pip install ffmpeg-python > /dev/null 2>&1
# packages required to show video
!pip install pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1


In [ ]:
env_id = "LunarLander-v2" #@param

In [ ]:
import gymnasium as gym

env = gym.make(env_id)
observation, info = env.reset(seed=42)
for _ in range(1000):
   action = env.action_space.sample()  # this is where you would insert your policy
   observation, reward, terminated, truncated, info = env.step(action)

   if terminated or truncated:
      observation, info = env.reset()

print(observation, reward, terminated, truncated, info)

In [ ]:
# Initialize display and import function to show videos
from gymnasium.utils.save_video import save_video
import rlberry.colab_utils.display_setup
from rlberry.colab_utils.display_setup import show_video

def get_env(for_render=False):
  if not for_render:
    return gym.make(env_id)
  else:
    return gym.make(env_id, render_mode="rgb_array_list")

def render_random_policy():
  env = get_env(for_render=True)
  state, _ = env.reset()
  step_starting_index = 0
  episode_index = 0
  for step_index in range(500):
    if episode_index > 0: # show only one episode
      break
    # random action
    action = env.action_space.sample()
    next_state, reward, terminated, truncated, info = env.step(action)
    # end of episode
    if terminated or truncated:
        save_video(
          env.render(),
          "videos",
          fps=env.metadata["render_fps"],
          step_starting_index=step_starting_index,
          episode_index=episode_index
        )
        step_starting_index = step_index + 1
        episode_index += 1
        next_state, _ = env.reset()
    state = next_state
  show_video("videos/rl-video-episode-0.mp4")


render_random_policy()


## Review of Main Concepts

### Markov Decision Processes and Value Functions

In reinforcement learning, an agent interacts with an enviroment by taking actions and observing rewards. Its goal is to learn a *policy*, that is, a mapping from states to actions, that maximizes the amount of reward it gathers.

The enviroment is modeled as a __Markov decision process (MDP)__, defined by a set of states $\mathcal{S}$, a set of actions $\mathcal{A}$, a reward function $r(s, a)$ and transition probabilities $P(s'|s,a)$. When an agent takes action $a$ in state $s$, it receives a random reward with mean $r(s,a)$ and makes a transion to a state $s'$ distributed according to $P(s'|s,a)$.

A __policy__ $\pi$ is such that $\pi(a|s)$ gives the probability of choosing an action $a$ in state $s$. __If the policy is deterministic__, we denote by $\pi(s)$ the action that it chooses in state $s$. We are interested in finding a policy that maximizes the value function $V^\pi$, defined as

$$
V^\pi(s) = \sum_{a\in \mathcal{A}} \pi(a|s) Q^\pi(s, a),
\quad \text{where} \quad
Q^\pi(s, a) = \mathbf{E}\left[ \sum_{t=0}^\infty \gamma^t r(S_t, A_t)  \Big| S_0 = s, A_0 = a\right].
$$
and represents the mean of the sum of discounted rewards gathered by the policy $\pi$ in the MDP, where $\gamma \in [0, 1[$ is a discount factor ensuring the convergence of the sum.

The __action-value function__ $Q^\pi$ is the __fixed point of the Bellman operator $T^\pi$__:

$$
Q^\pi(s, a) = T^\pi Q^\pi(s, a)
$$
where, for any function $f: \mathcal{S}\times\mathcal{A} \to \mathbb{R}$
$$
T^\pi f(s, a) =  r(s, a) + \gamma \sum_{s'} P(s'|s,a) \left(\sum_{a'}\pi(a'|s')f(s',a')\right)
$$


The __optimal value function__, defined as $V^*(s) = \max_\pi V^\pi(s)$ can be shown to satisfy $V^*(s) = \max_a Q^*(s, a)$, where $Q^*$ is the __fixed point of the optimal Bellman operator $T^*$__:

$$
Q^*(s, a) = T^* Q^*(s, a)
$$
where, for any function $f: \mathcal{S}\times\mathcal{A} \to \mathbb{R}$
$$
T^* f(s, a) =  r(s, a) + \gamma \sum_{s'} P(s'|s,a) \max_{a'} f(s', a')
$$
and there exists an __optimal policy__ which is deterministic, given by $\pi^*(s) \in \arg\max_a Q^*(s, a)$.



### Value iteration

If both the reward function $r$ and the transition probablities $P$ are known, we can compute $Q^*$ using value iteration, which proceeds as follows:

1. Start with arbitrary $Q_0$, set $t=0$.
2. Compute $Q_{t+1}(s, a) = T^*Q_t(s,a)$ for every $(s, a)$.
3. If $\max_{s,a} | Q_{t+1}(s, a) -  Q_t(s,a)| \leq \varepsilon$, return $Q_{t}$. Otherwise, set $t \gets t+1$ and go back to 2.

The convergence is guaranteed by the contraction property of the Bellman operator, and $Q_{t+1}$ can be shown to be a good approximation of $Q^*$ for small epsilon.

__Question__: Can you bound the error $\max_{s,a} | Q^*(s, a) -  Q_t(s,a)|$ as a function of $\gamma$ and $\varepsilon$?


### Q-Learning

In value iteration, we need to know $r$ and $P$ to implement the Bellman operator. When these quantities are not available, we can approximate $Q^*$ using *samples* from the environment with the Q-Learning algorithm.

Q-Learning with __$\varepsilon$-greedy exploration__ proceeds as follows:

1. Start with arbitrary $Q_0$, get starting state $s_0$, set $t=0$.
2. Choosing action $a_t$:
  * With probability $\varepsilon$ choose $a_t$ randomly (uniform distribution)  
  * With probability $1-\varepsilon$, choose $a_t \in \arg\max_a Q_t(s_t, a)$.
3. Take action $a_t$, observe next state $s_{t+1}$ and reward $r_t$.
4. Compute error $\delta_t = r_t + \gamma \max_a Q_t(s_{t+1}, a) - Q_t(s_t, a_t)$.
5. Update
  * $Q_{t+1}(s, a) = Q_t(s, a) + \alpha_t(s,a) \delta_t$,  __if $s=s_t$ and $a=a_t$__
  * $Q_{t+1}(s, a) = Q_{t}(s, a)$ otherwise.

Here, $\alpha_t(s,a)$ is a learning rate that can depend, for instance, on the number of times the algorithm has visited the state-action pair $(s, a)$.

### Fitted Q-Iteration

Given a datset $(s_i, a_i, r_i, s_i')$ of (states, actions, rewards, next states), the Fitted Q-Iteration (FQI) algorithm proceeds as follows:


* We start from a $Q$ function $Q_0 \in \mathcal{F}$, where $\mathcal{F}$ is a function space;
* At every iteration $k$, we compute $Q_{k+1}$ as:

$$
Q_{k+1}\in\arg\min_{f\in\mathcal{F}} \frac{1}{2}\sum_{i=1}^N
\left(
  f(s_i, a_i) - y_i^k
\right)^2 + \lambda \Omega(f)
$$
where $y_i^k = r_i + \gamma \max_{a'}Q_k(s_i', a')$, $\Omega(f)$ is a regularization term and $\lambda > 0$ is the regularization coefficient.


Consider FQI with *linear* function approximation. That is, for a given feature map $\phi : S \rightarrow \mathbb{R}^d$, we consider a parametric family of $Q$ functions $Q_\theta(s,a) = \phi(s)^T\theta_a$ for $\theta_a\in\mathbb{R}^d$. Suppose we are applying FQI on a given dataset of $N$ tuples of the form $(s_i, a_i, r_i, s_i')$ and we are at the $k$-th iteration. Let $\theta_k \in\mathbb{R}^{d \times A}$ be our current parameter. The *closed-form* update to find $\theta_{k+1}$, using $\frac{1}{2}\sum_a ||\theta_a||_2^2$ as regularization is found by solving the linear system:


$$
(M_a + \lambda I) \theta_a = b_a
$$

where:

* $M_a = \sum_{i} \mathbb{1}\{a_i = a\}\phi(s_i)\phi(s_i)^T$ is a (d, d) matrix.
* $b_a = \sum_{i} \mathbb{1}\{a_i = a\}\phi(s_i) y_i$ is a (d, 1) matrix.
* $y_i = r_i + \gamma \max_{a'} \phi(s_i', a)^T \theta_a $
* $\mathbb{1}\{a_i = a\}$ is 1 if $a_i=a$ and 0 otherwise.

### Deep Q-Learning


Deep Q-Learning uses a neural network to approximate $Q$ functions. Hence, we usually refer to this algorithm as DQN (for *deep Q network*).

The parameters of the neural network are denoted by $\theta$.
*   As input, the network takes a state $s$,
*   As output, the network returns $Q(s, a, \theta)$, the value of each action $a$ in state $s$, according to the parameters $\theta$.


The goal of Deep Q-Learning is to learn the parameters $\theta$ so that $Q(s, a, \theta)$ approximates well the optimal $Q$-function $Q^*(s, a)$.

In addition to the network with parameters $\theta$, the algorithm keeps another network with the same architecture and parameters $\theta^-$, called **target network**.

The algorithm works as follows:

1.   At each time $t$, the agent is in state $s_t$ and has observed the transitions $(s_i, a_i, r_i, s_i')_{i=1}^{t-1}$, which are stored in a **replay buffer**.

2.  Choose action $a_t = \arg\max_a Q(s_t, a)$ with probability $1-\varepsilon_t$, and $a_t$=random action with probability $\varepsilon_t$.

3. Take action $a_t$, observe reward $r_t$ and next state $s_t'$.

4. Add transition $(s_t, a_t, r_t, s_t')$ to the **replay buffer**.

4.  Sample a minibatch $\mathcal{B}$ containing $B$ transitions from the replay buffer. Using this minibatch, we define the loss:

$$
L(\theta) = \sum_{(s_i, a_i, r_i, s_i') \in \mathcal{B}}
\left[
Q(s_i, a_i, \theta) -  y_i
\right]^2
$$
where the $y_i$ are the **targets** computed with the **target network** $\theta^-$:

$$
y_i = r_i + \gamma \max_{a'} Q(s_i', a', \theta^-).
$$

5. Update the parameters $\theta$ to minimize the loss, e.g., with gradient descent (**keeping $\theta^-$ fixed**):
$$
\theta \gets \theta - \eta \nabla_\theta L(\theta)
$$
where $\eta$ is the optimization learning rate.

6. Every $N$ transitions ($t\mod N$ = 0), update target parameters: $\theta^- \gets \theta$.

7. $t \gets t+1$. Stop if $t = T$, otherwise go to step 2.